In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print("mnist dataset: train=%s test=%s" % (x_train.shape, x_test.shape))
# print("x_test[0] =", x_test[0])

mnist dataset: train=(60000, 28, 28) test=(10000, 28, 28)


In [3]:
x_train, x_test = x_train / 255.0, x_test / 255.0  # normalizing
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax'),
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [4]:
class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        self.model.stop_training = logs.get('accuracy') > 0.99

In [5]:
model.fit(x_train,
          y_train,
          epochs=50,
          callbacks=[myCallback()])
model.evaluate(x_test, y_test)

Train on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 8s 129us/sample - loss: 0.2628 - accuracy: 0.9217
Epoch 2/50
60000/60000 [==============================] - 7s 123us/sample - loss: 0.1383 - accuracy: 0.9612
Epoch 3/50
60000/60000 [==============================] - 7s 124us/sample - loss: 0.1095 - accuracy: 0.9690
Epoch 4/50
60000/60000 [==============================] - 7s 123us/sample - loss: 0.0923 - accuracy: 0.9741
Epoch 5/50
60000/60000 [==============================] - 8s 131us/sample - loss: 0.0823 - accuracy: 0.9774
Epoch 6/50
60000/60000 [==============================] - 8s 128us/sample - loss: 0.0747 - accuracy: 0.9794
Epoch 7/50
60000/60000 [==============================] - 8s 131us/sample - loss: 0.0688 - accuracy: 0.9810 - loss: 0.0688 - accuracy: 0.98
Epoch 8/50
60000/60000 [==============================] - 8s 131us/sample - loss: 0.0602 - accuracy: 0.9833
Epoch 9/50
60000/60000 [==============================] - 8s 137us/sample - loss:

[0.11458113379694249, 0.9821]

In [ ]:
# save tflite model
tf.saved_model.save(model, 'models/digits_mnist')
converter = tf.lite.TFLiteConverter.from_saved_model('models/digits_mnist')
lite_model = converter.convert()
open('models/digits_mnist.tflite', 'wb').write(lite_model)